In [6]:
# Install folium for map visualization
!pip install folium

# Import libraries
import pandas as pd
import folium
from folium.plugins import HeatMap, MarkerCluster
import unicodedata


In [7]:
# Part 1: Load and Clean Data for EV Charging Stations
# ========================================================
# This section handles the loading and cleaning of the EV charging station data.

# Step 1.1: Load the EV charging station dataset
# Load the EV charging station data from a specified path
df = pd.read_csv('/Users/zactseng/Documents/ECE 1724 Bio/Project/EV_Charging_Stations_Placement_Optimization/Quebec_EV_charging_station.csv')

# Step 1.2: Normalize 'City' column by removing accents and converting to lowercase
# This ensures that cities with accents are properly filtered (e.g., "Montréal" becomes "montreal")
df['City'] = df['City'].apply(lambda x: unicodedata.normalize('NFKD', x).encode('ASCII', 'ignore').decode('utf-8').lower())

# Step 1.3: Filter for rows where 'City' is 'montreal' and select specific columns
# Only include Montreal EV charging stations with relevant columns
montreal_df = df[df['City'] == 'montreal']
montreal_df = montreal_df[['City', 'Latitude', 'Longitude', 'Price', 'Pricing']]

# Check if there is any data for Montreal before saving
if not montreal_df.empty:
    # Save the filtered dataset for later use
    output_path = 'Montreal_EV_charging_stations.csv'
    montreal_df.to_csv(output_path, index=False)
else:
    print("No data found for 'Montreal' in the dataset after cleaning.")

In [8]:
# Part 2: Visualize EV Charging Stations with Price-Based Markers
# ========================================================
# This section adds markers for each EV charging station in Montreal,
# with different colors to represent different pricing levels.

# Initialize a map centered on Montreal
montreal_map = folium.Map(location=[45.5017, -73.5673], zoom_start=12)

# Step 2.1: Add markers for each charging station with color coding based on price
# Orange for Price > 1, Blue for Price <= 1
for index, row in montreal_df.iterrows():
    marker_color = "orange" if row['Price'] > 1 else "blue"

    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"Price: {row['Price']}, Pricing: {row['Pricing']}",
        icon=folium.Icon(color=marker_color, icon="bolt", prefix="fa")
    ).add_to(montreal_map)

montreal_map

In [9]:
# Part 3: Separate Heat Map for Traffic Flow Density
# ========================================================
# This section creates a separate heat map to represent the density of traffic flows
# across different locations in Montreal, so it does not overlap with the EV charging map.

# Step 3.1: Load the traffic flow dataset and clean it
# Load traffic flow data from a specified path
traffic_df = pd.read_csv('/Users/zactseng/Documents/ECE 1724 Bio/Project/EV_Charging_Stations_Placement_Optimization/Montreal_traffic_flow.csv')

# Drop rows with missing values in latitude, longitude, or traffic count
traffic_df = traffic_df.dropna(subset=['latitude', 'longitude', 'average_vehicle_count'])

# Step 3.2: Prepare data for the heat map (latitude, longitude, intensity)
# This data format is required for HeatMap to visualize traffic flow intensity
heat_data = traffic_df[['latitude', 'longitude', 'average_vehicle_count']].values.tolist()

# Initialize a new map for the heat map to avoid overlap
montreal_heat_map = folium.Map(location=[45.5017, -73.5673], zoom_start=12)

# Step 3.3: Add HeatMap to visualize traffic density with a color gradient for intensity
HeatMap(
    heat_data,
    radius=15,                   # Larger radius for denser coverage
    blur=10,                     # Set blur for smoother transitions
    min_opacity=0.2,             # Minimum opacity for less dense areas
    max_opacity=0.8,             # Maximum opacity for denser areas
    max_val=traffic_df['average_vehicle_count'].max(), # Normalize colors by max traffic count
    gradient={0.2: 'blue', 0.4: 'cyan', 0.6: 'lime', 0.8: 'orange', 1: 'red'} # Custom gradient
).add_to(montreal_heat_map)

montreal_heat_map

/var/folders/sq/ksk005sx1k1b1v7q1kfnjlhc0000gn/T/ipykernel_6142/2431988086.py:21: UserWarning: The `max_val` parameter is no longer necessary. The largest intensity is calculated automatically.
  HeatMap(


In [10]:
# Part 4: Cluster Map for Traffic Flow Density
# ========================================================
# This section visualizes traffic flow density using a MarkerCluster.
# Clusters group nearby traffic points, showing dense regions with marker counts.

# Initialize a new map centered on Montreal for the cluster map
montreal_cluster_map = folium.Map(location=[45.5017, -73.5673], zoom_start=12)

# Step 4.1: Initialize a MarkerCluster to group traffic flow markers
marker_cluster = MarkerCluster().add_to(montreal_cluster_map)

# Step 4.2: Add traffic data points to the cluster, showing average vehicle count on popup
for index, row in traffic_df.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"Average Vehicle Count: {row['average_vehicle_count']}"
    ).add_to(marker_cluster)

montreal_cluster_map